In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 
     |████████████████████████████████| 120 kB 53.6 MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [9]:
import pandas as pd
import numpy as np

import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel
from keras.models import Sequential, Model

from tqdm.notebook import tqdm
tqdm.pandas()

import multiprocessing as mp

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/allData-19-08-27.ta'
df = pd.read_csv(data, header=None, names=['news'])
df.head(), len(df)

(                                                news
 0  அமைச்சின் பணிகளை முன்னெடுப்பதற்கு கௌர அமைச்சர்...
 1  இவ்வமைச்சு இல 40  புத்கமுவ வீதி  இராஜகிரிய எனு...
 2  தேசிய மொழிகள் மற்றும் சமூக ஒருமைப்பாட்டு அமைச்...
 3   தேசிய மொழிகள்  சமூக ஒருமைப்பாடு தொடர்பான கொள்...
 4   குறித்த தேசிய நோக்கங்களை எய்தும் பொருட்டு தேச..., 1727888)

In [5]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
def get_vectors(news):
  input_ids = tf.constant(tokenizer.encode(news, 
                                           add_special_tokens = True,
                                           max_length = 512, 
                                           truncation = True, 
                                           padding = "max_length"))[None, :]
  outputs = model(input_ids)

  last_hidden_states = outputs[0]
  cls_token = last_hidden_states[0]

  return cls_token

In [10]:
p = mp.Pool(mp.cpu_count())
df['vec'] = p.map(get_vectors, df['news'])

KeyboardInterrupt: ignored

In [ ]:
# df['vec'] = df['news'].progress_apply(get_vectors)

In [ ]:
base_url = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/vec.csv'
df.to_csv(base_url, encoding='utf-8')